# Scrape Data from Shopee with beautifulsoup and selenium
### Data เพื่อการทำ Marketing ของผู้ขาย ซึ่งประกอบไปด้วย
- ชื่อสินค้าสินค้า ราคาสินค้าที่ขายจริง ยอดขายแต่ละสินค้า จังหวัดที่เป็นต้นทางของสินค้า เวลาที่คอมเม้น คอมเม้นสินค้า

!pip install selenium
!pip install bs4
!pip install pandas

# Code ที่ต้อง Import

In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

In [11]:
# Set up Chrome options
options = Options()
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"

# Specify the path to ChromeDriver (NOT geckodriver, which is for Firefox)
service = Service(r'C:\Users\earns\Desktop\Shopee_Web_Scarping\chromedriver.exe')

# Initialize WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)

# Open lazada website
driver.get('https://www.lazada.co.th/products/90-biore-uv-aqua-rich-watery-gel-spf50-pa-90ml-i228851585.html?spm=a2o4m.searchlist.list.11.592a32030G8dMB')

In [50]:
#เลือกภาษาไทย Selenium
thai_button = driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
#time.sleep(3)

In [47]:
#กรอก email pass
username_field = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/div/div[2]/form/div[1]/div[1]/input')
username_field.send_keys(Keys.CONTROL + 'a')  # เลือกข้อความทั้งหมด
username_field.send_keys(Keys.BACKSPACE)  # ลบข้อความที่เลือก
username_field.send_keys("earn.sirikorn2002@gmail.com")

password_field = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/div/div[2]/form/div[2]/div[1]/input')
password_field.send_keys(Keys.CONTROL + 'a')  # เลือกข้อความทั้งหมด
password_field.send_keys(Keys.BACKSPACE)  # ลบข้อความที่เลือก
password_field.send_keys("earn56861212N")

In [48]:
#login
login_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div/div/div/div[2]/div/div[2]/form/button')
login_button.click()

In [ ]:
#ปิดโฆษณา Selenium
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()

In [63]:
#Search ใน Search bar ของ Shopee
search = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[2]/div/div[2]/div/form/div/div[1]/input[1]')
search.send_keys('Biore UV Aqua Rich Watery Gel')

In [64]:
#กด Enter
search.send_keys(Keys.ENTER)

In [65]:
#เลือกสินค้า 1 ชิ้น
select_button = driver.find_element(By.XPATH,  '/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[1]/div/div')
select_button.click()

In [12]:
#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.zoom='10%'")

In [ ]:
#เลือกเป็น sort จาก 1 ดาว

# Scraping Process

In [56]:
data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup

In [ ]:
soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div") #css selector

In [58]:
el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")

In [ ]:
len(el) #Component Comment ที่เก็บมาได้

In [60]:
e = el[0] #แทนกล่องของสินค้าแต่ละกล่อง

# Scrape comment

In [ ]:
#ดึงข้อมูล comment
e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content").text.strip()
##module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div:nth-child(1) > div.item-content > div.content

# Scrap ดาว

In [ ]:
# ดึงดาว
stars=e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
# กำหนด src สำหรับการนับคะแนน
src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

# เก็บค่า rating
rating = 0
for star in stars:
    if star['src'] == src_plus_one:
        rating += 1
    elif star['src'] == src_plus_zero:
        rating += 0

# แสดงผล rating
print(f"Star Rating: {rating}")

# Scarp เวลา

In [ ]:
e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span').text.strip()

# Scarp ชื่อ

In [ ]:
e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)').text.strip()

# Scrape ราคา

In [ ]:
#ราคา A baht - B baht, ราคาเต็ม
e.select_one("div.col-xs-2-4> a> div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()

In [ ]:
#ราคาลดแล้ว
p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
if p == '':
    print(1/0)
p

In [ ]:
#ราคารวมการตัด
try:
    p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
    if p == '':
        print(1/0)
except:
    p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
p

In [ ]:
#ตัด - และเลือกเลขด้านหน้า Range
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

In [ ]:
#รวม
try:
    p = e.select_one("div.col-xs-2-4 > a > div > div > div:nth-child(2) > div:nth-child(2)> div:nth-child(2)").text.strip()
except:
    p = e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1)").text.strip()
p = p.replace(",", "").replace("฿", "").replace(" ", "")
try:
    start_price = float(p)
except:
    price = p.rpartition('-')[0]
    start_price = float(price)

start_price

# Scrape ยอดขาย

In [ ]:
sales = e.select_one("div.col-xs-2-4 > a > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(3) > div:nth-child(3)").text.strip().replace("พัน","000").replace("ล้าน","000000")
try:
    sales = float(re.sub('\D','',sales))
except:
    sales = 0
sales

# Scrape จังหวัด

In [ ]:
#จังหวัด
e.select_one("div.col-xs-2-4 > a:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(4)").text.strip()

# Scrape ข้อมูลทั้งหมดที่ต้องการ

ทำเป็น list ของในหน้านั้น ๆ ที่กำลังดูอยู่ใน Web Shopee
นำ Code ต่าง ๆ มารวมกัน

In [71]:
records=[]
data = driver.page_source
soup = bs4.BeautifulSoup(data)
el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
# Define star rating image sources
src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

for e in el:
    # Extract review text
    review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
    review_text = review.text.strip() if review else ""

    # Extract and calculate rating
    stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
    rating = 0
    if stars:
        for star in stars.find_all('img'):
            if star['src'] == src_plus_one:
                rating += 1
            elif star['src'] == src_plus_zero:
                rating += 0

    # Extract review date
    date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
    date = date_element.text.strip() if date_element else ""

    # Extract reviewer name
    name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
    name = name_element.text.strip() if name_element else ""

    # Append the extracted data to records
    records.append([name, rating, date, review_text])

In [ ]:
len(records)

In [ ]:
#ทำเป็น DataFrame
df = pd.DataFrame(records, columns=['name','rating','date','review_text']) # ใส้ชื่อ Columns ทั้งหมด
df

In [ ]:
#ใช้ Selenium ในการกดปุ่มเพื่อไปหน้าถัดไป
next_button = driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/div/div/div[2]/div[2]/div[3]/div/button[8]')
next_button.click()

In [18]:
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
records=[]
for i in range(25):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
    # Define star rating image sources
    src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
    src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

    for e in el:
        # Extract review text
        review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
        review_text = review.text.strip() if review else ""

        # Extract and calculate rating
        stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
        rating = 0
        if stars:
            for star in stars.find_all('img'):
                if star['src'] == src_plus_one:
                    rating += 1
                elif star['src'] == src_plus_zero:
                    rating += 0

        # Extract review date
        date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
        date = date_element.text.strip() if date_element else ""

        # Extract reviewer name
        name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
        name = name_element.text.strip() if name_element else ""

        # Append the extracted data to records
        records.append([name, rating, date, review_text])

    wait = WebDriverWait(driver, 20)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')))
    next_button.click()
    time.sleep(15)
    print(i)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [23]:
#ทำเป็น DataFrame
# สร้าง DataFrame และเพิ่มลำดับแถว
df = pd.DataFrame(records, columns=['name', 'rating', 'date', 'review_text'])

# เพิ่มคอลัมน์ลำดับแถว (Row Index) โดยใช้ range
df['no'] = range(1, len(df) + 1)

# แสดง DataFrame ที่มีลำดับแถว
df = df[['no', 'name', 'rating', 'date', 'review_text']]
df

,no,name,rating,date,review_text
0,1,จ***.,5,2 สัปดาห์ก่อน,ร้านค้าจัดส่งสินค้าได้รวดเร็วมาก การแพ็คสินค้า...
1,2,P***i,5,23 มิ.ย. 2024,สะใจมากกันแดดขวดนี้ปริมาณ 90 ml. กดมาได้ในราคา...
2,3,นิตยา เ.,5,4 สัปดาห์ก่อน,"สูตรที่ยาวนาน, อ่อนโยนต่อผิวแพ้ง่าย, รู้สึกสดช..."
3,4,g***t,5,18 ก.ค. 2024,สินค้า คุณภาพดีสมราคาดีเลย การจัดส่ง ไม่ช้ามาก...
4,5,พิณพินัช บ.,5,04 มิ.ย. 2024,"อ่อนโยนต่อผิวที่แพ้ง่าย, ซึมเข้าสู่ผิวได้อย่าง..."
...,...,...,...,...,...
120,121,T***N,5,20 พ.ย. 2023,ได้รับสินค้าถูกต้อง ตรงตามที่สั่ง ส่งสินค้าให้...
121,122,นาง ผ.,5,25 ก.ค. 2024,"ไม่ทำให้ผิวขาวและหมองคล้ำ, ไม่ทำให้เกิดสิว, ใช..."
122,123,โ***.,5,04 ก.ค. 2024,สินค้าคถณภาพดีมากครับ ราคาถูกกว่าในห้าง การแพ้...
123,124,S***.,5,06 พ.ค. 2024,"ใช้ง่ายและผสมเข้ากับผิวได้ดี, ซึมเข้าสู่ผิวได้..."


In [24]:
df.to_csv('Biore UV Aqua Rich Watery Gel_5star.csv', index=False)

# สูตรที่สามารถนำไปใช้ได้

In [97]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import bs4
import time
import re

In [98]:
# ตั้งแต่เริ่มจนจบ
# Set up Chrome options
options = Options()
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"
# Specify the path to ChromeDriver (NOT geckodriver, which is for Firefox)
service = Service(r'C:\Users\earns\Desktop\Shopee_Web_Scarping\chromedriver.exe')
# Initialize WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)
# Open lazada website
driver.get('https://www.lazada.co.th/products/90-biore-uv-aqua-rich-watery-gel-spf50-pa-90ml-i228851585.html?spm=a2o4m.searchlist.list.11.592a32030G8dMB')


In [ ]:

###

#Zoom out เพื่อให้ load ทั่วหน้าจอ
driver.execute_script("document.body.style.zoom='10%'")

###
# การ Scrape ข้อมูลจากหลายหน้าของ Web Shopee
records=[]
for i in range(2):
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)
    el=soup.select("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div")
    # Define star rating image sources
    src_plus_one = "//img.lazcdn.com/g/tps/tfs/TB19ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"
    src_plus_zero = "//img.lazcdn.com/g/tps/tfs/TB18ZvEgfDH8KJjy1XcXXcpdXXa-64-64.png"

    for e in el:
        # Extract review text
        review = e.select_one("#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.item-content > div.content")
        review_text = review.text.strip() if review else ""

        # Extract and calculate rating
        stars = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > div')
        rating = 0
        if stars:
            for star in stars.find_all('img'):
                if star['src'] == src_plus_one:
                    rating += 1
                elif star['src'] == src_plus_zero:
                    rating += 0

        # Extract review date
        date_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.top > span')
        date = date_element.text.strip() if date_element else ""

        # Extract reviewer name
        name_element = e.select_one('#module_product_review > div > div > div:nth-child(3) > div.mod-reviews > div > div.middle > span:nth-child(1)')
        name = name_element.text.strip() if name_element else ""

        # Append the extracted data to records
        records.append([name, rating, date, review_text])

    wait = WebDriverWait(driver, 20)
    next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]')))
    next_button.click()

records

In [ ]:
len(records)

In [ ]:
df = pd.DataFrame(records, columns=['itemname','price','qty','province'])
df

In [91]:
df.to_csv('output.csv', index=False)